Dataset

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dasgroup/rba-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.10G/1.10G [00:06<00:00, 192MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dasgroup/rba-dataset/versions/1


In [5]:
import pandas as pd

data = pd.read_csv(path + '/rba-dataset.csv')
print(data.head())

   index          Login Timestamp              User ID  Round-Trip Time [ms]  \
0      0  2020-02-03 12:43:30.772 -4324475583306591935                   NaN   
1      1  2020-02-03 12:43:43.549 -4324475583306591935                   NaN   
2      2  2020-02-03 12:43:55.873 -3284137479262433373                   NaN   
3      3  2020-02-03 12:43:56.180 -4324475583306591935                   NaN   
4      4  2020-02-03 12:43:59.396 -4618854071942621186                   NaN   

      IP Address Country    Region       City     ASN  \
0    10.0.65.171      NO         -          -   29695   
1   194.87.207.6      AU         -          -   60117   
2  81.167.144.58      NO  Vestland  Urangsvag   29695   
3  170.39.78.152      US         -          -  393398   
4      10.0.0.47      US  Virginia    Ashburn  398986   

                                   User Agent String  \
0  Mozilla/5.0  (iPhone; CPU iPhone OS 13_4 like ...   
1  Mozilla/5.0  (Linux; Android 4.1; Galaxy Nexus...   
2  Mozil

Data Preprocessing

In [10]:
import requests

unique_countries = data['Country'].unique()
country_code_to_name = {}

for country_code in unique_countries:
    try:
        url = f"https://restcountries.com/v3.1/alpha/{country_code}"
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        country_data = response.json()[0]  # Access the first element of the list
        country_name = country_data.get("name", {}).get("common") # Extract common name

        if country_name:
            country_code_to_name[country_code] = country_name
        else:
            print(f"Could not find a common name for country code: {country_code}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {country_code}: {e}")
    except (IndexError, KeyError) as e:
        print(f"Error processing data for {country_code}: {e}")

country_code_to_name

{'NO': 'Norway',
 'AU': 'Australia',
 'US': 'United States',
 'ID': 'Indonesia',
 'BR': 'Brazil',
 'PL': 'Poland',
 'IN': 'India',
 'DK': 'Denmark',
 'DE': 'Germany',
 'BD': 'Bangladesh',
 'MD': 'Moldova',
 'NL': 'Netherlands',
 'RO': 'Romania',
 'GB': 'United Kingdom',
 'AR': 'Argentina',
 'FI': 'Finland',
 'CZ': 'Czechia',
 'CA': 'Canada',
 'FR': 'France',
 'TR': 'Turkey',
 'RU': 'Russia',
 'CL': 'Chile',
 'BZ': 'Belize',
 'IR': 'Iran',
 'UA': 'Ukraine',
 'EG': 'Egypt',
 'CH': 'Switzerland',
 'IS': 'Iceland',
 'BG': 'Bulgaria',
 'TH': 'Thailand',
 'IT': 'Italy',
 'CO': 'Colombia',
 'PH': 'Philippines',
 'MX': 'Mexico',
 'CN': 'China',
 'VN': 'Vietnam',
 'MY': 'Malaysia',
 'PK': 'Pakistan',
 'ES': 'Spain',
 'IL': 'Israel',
 'SI': 'Slovenia',
 'NG': 'Nigeria',
 'CY': 'Cyprus',
 'LU': 'Luxembourg',
 'HU': 'Hungary',
 'VG': 'British Virgin Islands',
 'KR': 'South Korea',
 'GY': 'Guyana',
 'LV': 'Latvia',
 'NZ': 'New Zealand',
 'MW': 'Malawi',
 'LB': 'Lebanon',
 'PA': 'Panama',
 'DO': 'Do

In [11]:
# Extracting day, time from timestamp
data['Login DateTime'] = pd.to_datetime(data['Login Timestamp'])
data['Year'] = data['Login DateTime'].dt.year
data['Month'] = data['Login DateTime'].dt.month
data['Day'] = data['Login DateTime'].dt.day
data['Hour'] = data['Login DateTime'].dt.hour
data['Minute'] = data['Login DateTime'].dt.minute
data['Second'] = data['Login DateTime'].dt.second
data['Millisecond'] = data['Login DateTime'].dt.microsecond // 1000

In [12]:
# Converting Booleans To Integers
import numpy as np

data['Is Account Takeover'] = data['Is Account Takeover'].astype(np.uint8)
data['Is Attack IP'] = data['Is Attack IP'].astype(np.uint8)
data['Login Successful'] = data['Login Successful'].astype(np.uint8)

In [13]:
# Converting Strings To Integers
data['User Agent String'], _ = pd.factorize(data['User Agent String'])
data['Browser Name and Version'], _ = pd.factorize(data['Browser Name and Version'])
data['OS Name and Version'], _ = pd.factorize(data['OS Name and Version'])

In [14]:
# Converting IP Addresses To Integers
import ipaddress

def ip_to_int(ip):
    return int(ipaddress.ip_address(ip))

data['IP Address'] = data['IP Address'].apply(ip_to_int)

In [15]:
# Dropping Unneeded Columns
data = data.drop(columns=['Login Timestamp', 'index', 'Round-Trip Time [ms]'])

In [16]:
# Encoding Categorical & Numerical Variables
categorical_cols = ['Country', 'Region', 'City', 'Device Type']
numeric_cols = ['ASN', 'Year', 'Month', 'Day' ,'Hour', 'Minute', 'Second', 'Millisecond', 'User ID', 'IP Address', 'User Agent String', 'Browser Name and Version', 'OS Name and Version']

In [17]:
# Splitting The Dataset Into Train/Test
from sklearn.model_selection import train_test_split

features = data.drop(['Is Attack IP', 'Is Account Takeover'], axis=1)
labels = data[['Is Attack IP', 'Is Account Takeover']]
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

Feature Extraction using PCA

In [18]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Combine training and testing data for PCA
X_combined = pd.concat([X_train, X_test])

# Select numerical features for PCA
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols)
    ])

X_processed = preprocessor.fit_transform(X_combined)

# Apply PCA
pca = PCA(n_components=0.95) # Keep components explaining 95% of the variance
X_pca = pca.fit_transform(X_processed)

# Split the PCA-transformed data back into training and testing sets
X_train_pca = X_pca[:len(X_train)]
X_test_pca = X_pca[len(X_train):]

print(f"Original number of features: {X_processed.shape[1]}")
print(f"Number of features after PCA: {X_pca.shape[1]}")

Original number of features: 13
Number of features after PCA: 11


Anomaly detection using Autoencoder

In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define Autoencoder model
def create_autoencoder(input_dim):
  input_layer = keras.Input(shape=(input_dim,))

  # Optimized encoder layers
  encoded = layers.Dense(128, activation='relu')(input_layer)
  encoded = layers.BatchNormalization()(encoded)  # Batch normalization
  encoded = layers.Dropout(0.2)(encoded)  # Dropout for regularization
  encoded = layers.Dense(64, activation='relu')(encoded)
  encoded = layers.BatchNormalization()(encoded)
  encoded = layers.Dropout(0.2)(encoded)
  encoded = layers.Dense(32, activation='relu')(encoded)  # Bottleneck layer

  # Decoder layers
  decoded = layers.Dense(64, activation='relu')(encoded)
  decoded = layers.BatchNormalization()(decoded)
  decoded = layers.Dropout(0.2)(decoded)
  decoded = layers.Dense(128, activation='relu')(decoded)
  decoded = layers.BatchNormalization()(decoded)
  decoded = layers.Dropout(0.2)(decoded)
  decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)  # Output layer

  autoencoder = keras.Model(input_layer, decoded)
  return autoencoder


# Train Autoencoder
input_dim = X_train_pca.shape[1]
autoencoder = create_autoencoder(input_dim)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train_pca, X_train_pca, epochs=10, batch_size=50000, shuffle=False, validation_data=(X_test_pca, X_test_pca))

# Extract encoded features
encoded_features = autoencoder.predict(X_train_pca)
encoded_data = pd.DataFrame(encoded_features)

# Detect anomalies based on reconstruction error
reconstructions = autoencoder.predict(X_test_pca)
mse = np.mean(np.power(X_test_pca - reconstructions, 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.percentile(mse, 95)
anomalies = mse > threshold

print("Number of anomalies detected:", np.sum(anomalies))

ERROR: Could not find a version that satisfies the requirement tensorflow.keras (from versions: none)
ERROR: No matching distribution found for tensorflow.keras
Epoch 1/20
501/501 [==============================] - 50s 94ms/step - loss: 0.7608 - val_loss: 0.6738
Epoch 2/20
501/501 [==============================] - 46s 92ms/step - loss: 0.6919 - val_loss: 0.6741
Epoch 3/20
501/501 [==============================] - 46s 92ms/step - loss: 0.6855 - val_loss: 0.6731
Epoch 4/20
501/501 [==============================] - 46s 92ms/step - loss: 0.6818 - val_loss: 0.6709
Epoch 5/20
501/501 [==============================] - 45s 90ms/step - loss: 0.6781 - val_loss: 0.6685
Epoch 6/20
501/501 [==============================] - 45s 89ms/step - loss: 0.6742 - val_loss: 0.6667
Epoch 7/20
501/501 [==============================] - 46s 92ms/step - loss: 0.6713 - val_loss: 0.6661
Epoch 8/20
501/501 [==============================] - 45s 90ms/step - loss: 0.6696 - val_loss: 0.6654
Epoch 9/20
501/501 [===

Geolocation Mapping

In [49]:
import requests
import concurrent.futures
import time

api_key = 'fc2eaaeed3f144f7b0954d86e71aeb72'
request = 2100

# Get latitude and longitude
def get_lat_lon(city, country_code):
    """Gets latitude and longitude using OpenCage Geocoding API."""
    country_name = country_code_to_name.get(country_code, country_code)
    url = f'https://api.opencagedata.com/geocode/v1/json?q={city},{country_name}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        map_data = response.json()
        if map_data['results']:
            location = map_data['results'][0]['geometry']
            return location['lat'], location['lng']
    return None, None


# Process a single row
def process_row(row):
    lat, lon = get_lat_lon(row['City'], row['Country'])
    if lat and lon:
        return row['City'], lat, lon
    return row['City'], None, None


# Extract lesser data
geo_data = data[(data['City'] !='-') & (data['Region'] !='-')]
anomalies = [True] *request + [False] *(len(geo_data) - request)
anomalies_indices = geo_data.index[anomalies]
map_data = geo_data.loc[anomalies_indices]

# Add latitude and longitude columns to the DataFrame
map_data['Latitude'] = None
map_data['Longitude'] = None

# Use concurrent.futures to process rows in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(process_row, [row for _, row in map_data.iterrows()]))

# Update the DataFrame with results
for city, lat, lon in results:
    map_data.loc[map_data['City'] == city, 'Latitude'] = lat
    map_data.loc[map_data['City'] == city, 'Longitude'] = lon

print(map_data.head())

# Visualization using Plotly
import plotly.express as px

# Create a scatter plot of the IP addresses on a world map
fig = px.scatter_geo(map_data, lat='Latitude', lon='Longitude',
                     hover_name='City',
                     hover_data=['Country', 'Region', 'ASN'],
                     title='Geolocation Mapping of IP Addresses')

fig.show()

                User ID  IP Address Country    Region       City     ASN  \
2  -3284137479262433373  1369935930      NO  Vestland  Urangsvag   29695   
4  -4618854071942621186   167772207      US  Virginia    Ashburn  398986   
9  -3065936140549856249  1558015394      NO  Rogaland    Sandnes   29695   
11 -9080829243863829585  2620665939      NO     Viken   Gressvik   29695   
13 -4324475583306591935   785658794      NO     Viken     Klofta   41164   

    User Agent String  Browser Name and Version  OS Name and Version  \
2                   2                         2                    2   
4                   4                         3                    3   
9                   9                         7                    7   
11                 11                         9                    7   
13                 13                        11                    8   

   Device Type  ...          Login DateTime  Year  Month Day  Hour  Minute  \
2       mobile  ... 2020-02-03 1

Threat Region Tagging using Fuzzy C-Means Clustering

In [50]:
!pip install fuzzy-c-means
from fcmeans import FCM
from sklearn.preprocessing import StandardScaler

# Match encoded data for the newly extracted 2000 rows
encoded_map_data = map_data.iloc[:2000].copy()
encoded_map_data = encoded_map_data.reset_index(drop=True)

# Combine the encoded features with lat/lon data
clustering_data = map_data[['Latitude', 'Longitude']].reset_index(drop=True)
clustering_data = pd.concat([clustering_data, encoded_map_data], axis=1)
clustering_data.fillna(0, inplace=True)   # Handle missing values
print(clustering_data.head())

# Convert all column names to strings
clustering_data.columns = clustering_data.columns.astype(str)

# Ensure clustering_data contains only numeric columns for scaling
numeric_columns = clustering_data.select_dtypes(include=['float64', 'int64']).columns
numeric_data = clustering_data[numeric_columns]

# Scale data for FCM
scaler = StandardScaler()
clustering_data_scaled = scaler.fit_transform(numeric_data)

# Perform FCM clustering
fcm = FCM(n_clusters=5)  # Adjust number of clusters
fcm.fit(clustering_data_scaled)

# Get cluster labels for each data point
map_data['Cluster'] = fcm.predict(clustering_data_scaled)

# Analyzing clusters to identify threat regions
# Examining the distribution of attack IPs within each cluster
# Investigating the geographic location of clusters with high attack density
# Visualizing clusters on a map to highlight potential threat regions

# Count number of attack IPs in each cluster
cluster_attack_counts = map_data.groupby('Cluster')['Is Attack IP'].sum()

# Visualize clusters on a map
fig = px.scatter_geo(map_data, lat='Latitude', lon='Longitude', color='Cluster',
                     hover_name='City',
                     hover_data=['Country', 'Region', 'ASN', 'Is Attack IP', 'Is Account Takeover'],
                     title='Threat Region Tagging with Fuzzy C-Means Clustering')
fig.show()

<ipython-input-50-cc294f92616f>:12: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-50-cc294f92616f>:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.



    Latitude  Longitude       User ID    IP Address Country    Region  \
0  59.805840   5.175275 -3.284137e+18  1.369936e+09      NO  Vestland   
1  31.705975 -83.653559 -4.618854e+18  1.677722e+08      US  Virginia   
2  58.983132   6.394641 -3.065936e+18  1.558015e+09      NO  Rogaland   
3  59.218247  10.906389 -9.080829e+18  2.620666e+09      NO     Viken   
4  60.075343  11.139150 -4.324476e+18  7.856588e+08      NO     Viken   

        City       ASN  User Agent String  Browser Name and Version  ...  \
0  Urangsvag   29695.0                2.0                       2.0  ...   
1    Ashburn  398986.0                4.0                       3.0  ...   
2    Sandnes   29695.0                9.0                       7.0  ...   
3   Gressvik   29695.0               11.0                       9.0  ...   
4     Klofta   41164.0               13.0                      11.0  ...   

               Login DateTime    Year  Month  Day  Hour Minute  Second  \
0  2020-02-03 12:43:55.873000 

Geo-Behavioural Profiling using Naive Bayes classifier


In [51]:
from sklearn.naive_bayes import GaussianNB

X_nb = encoded_data
y_nb = data[['Is Attack IP', 'Is Account Takeover']][:len(encoded_data)]

# Split the data into training and testing sets
X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(X_nb, y_nb, test_size=0.2, random_state=42)

# Train a Naive Bayes classifier for 'Is Attack IP'
nb_attack_ip = GaussianNB()
nb_attack_ip.fit(X_train_nb, y_train_nb['Is Attack IP'])

# Train a Naive Bayes classifier for 'Is Account Takeover'
nb_account_takeover = GaussianNB()
nb_account_takeover.fit(X_train_nb, y_train_nb['Is Account Takeover'])

# Predict on the test set
y_pred_attack_ip = nb_attack_ip.predict(X_test_nb)
y_pred_account_takeover = nb_account_takeover.predict(X_test_nb)

In [52]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Evaluation metrics
print("Predictions for 'Is Attack IP':", y_pred_attack_ip)
print("Accuracy:", accuracy_score(y_test_nb['Is Attack IP'], y_pred_attack_ip))
print("Precision:", precision_score(y_test_nb['Is Attack IP'], y_pred_attack_ip))
print("Recall:", recall_score(y_test_nb['Is Attack IP'], y_pred_attack_ip))
print("F1 Score:", f1_score(y_test_nb['Is Attack IP'], y_pred_attack_ip))
print("Classification Report:\n", classification_report(y_test_nb['Is Attack IP'], y_pred_attack_ip))

print("\nPredictions for 'Is Account Takeover':", y_pred_account_takeover)
print("Accuracy:", accuracy_score(y_test_nb['Is Account Takeover'], y_pred_account_takeover))
print("Precision:", precision_score(y_test_nb['Is Account Takeover'], y_pred_account_takeover))
print("Recall:", recall_score(y_test_nb['Is Account Takeover'], y_pred_account_takeover))
print("F1 Score:", f1_score(y_test_nb['Is Account Takeover'], y_pred_account_takeover))
print("Classification Report:\n", classification_report(y_test_nb['Is Account Takeover'], y_pred_account_takeover))

Predictions for 'Is Attack IP': [0 0 0 ... 0 0 0]
Accuracy: 0.9042044675253239


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



Precision: 0.0
Recall: 0.0
F1 Score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Classification Report:
               precision    recall  f1-score   support

           0       0.90      1.00      0.95   4523810
           1       0.00      0.00      0.00    479273

    accuracy                           0.90   5003083
   macro avg       0.45      0.50      0.47   5003083
weighted avg       0.82      0.90      0.86   5003083


Predictions for 'Is Account Takeover': [0 0 0 ... 0 0 0]
Accuracy: 0.9999954028346122


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



Precision: 0.0
Recall: 0.0
F1 Score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   5003060
           1       0.00      0.00      0.00        23

    accuracy                           1.00   5003083
   macro avg       0.50      0.50      0.50   5003083
weighted avg       1.00      1.00      1.00   5003083



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.




Geographic Risk scoring

In [53]:
from operator import is_
risk_scores = []

def geographic_risk_score(data):
    """Assigns a geographic risk score based on cluster and attack probability."""
    for _, row in data.iterrows():
        cluster = row['Cluster']
        is_attack_ip = row['Is Attack IP']
        is_account_takeover = row['Is Account Takeover']

        # Proportion of attacks in the cluster
        risk = cluster_attack_counts[cluster] / data.shape[0]

        if is_account_takeover:
            risk *= 2     # Higher risk for account takeover
        if is_attack_ip:
            risk *= 1   # Moderate risk for attack IPs
        else:
            risk *= 0.5  # Low risk for non-attack IPs

        risk_scores.append(risk)
    return risk_scores


def assign_risk_category(risk_score):
    if risk_score >= 0.9:
        return "Very High Risk"
    elif risk_score >= 0.7:
        return "High Risk"
    elif risk_score >= 0.4:
        return "Medium Risk"
    else:
        return "Low Risk"

In [54]:
# Calculate risk scores
map_data['Risk Score'] = geographic_risk_score(map_data)
print(map_data[['City', 'Country', 'Cluster', 'Is Attack IP', 'Is Account Takeover', 'Risk Score']].head())

# Assign risk categories to the risk scores
risk_categories = [assign_risk_category(score) for score in risk_scores]

# Create a DataFrame for risk visualization
risk_data = map_data[['Latitude', 'Longitude']].copy()
risk_data['Risk Score'] = risk_scores
risk_data['Risk Category'] = risk_categories

         City Country  Cluster  Is Attack IP  Is Account Takeover  Risk Score
2   Urangsvag      NO        2             0                    0    0.003095
4     Ashburn      US        1             1                    0    0.029048
9     Sandnes      NO        2             0                    0    0.003095
11   Gressvik      NO        2             0                    0    0.003095
13     Klofta      NO        2             0                    0    0.003095


Visualization

In [55]:
import plotly.express as px

# Visualize risk scores on a map with improved color scale
fig = px.scatter_geo(risk_data,
                     lat='Latitude',
                     lon='Longitude',
                     color='Risk Category',
                     color_discrete_sequence=['green', 'yellow', 'orange', 'red'],
                     hover_data=['Latitude', 'Longitude', 'Risk Score', 'Risk Category'], # Add more info to hover
                     title='Geographic Risk Scoring',
                     labels={'Risk Category': 'Risk Level'}, # Customize axis labels
                     opacity=0.7 # Adjust point transparency
                     )

# Add controls for filtering
fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            buttons=list([
                dict(label="All Risk Levels",
                     method="update",
                     args=[{"visible": [True] * len(fig.data)},
                           {"title": "Geographic Risk Scoring - All Risk Levels"}]),
                dict(label="Very High Risk",
                     method="update",
                     args=[{"visible": [d.name == "Very High Risk" for d in fig.data]},
                           {"title": "Geographic Risk Scoring - Very High Risk"}]),
                dict(label="High Risk",
                     method="update",
                     args=[{"visible": [d.name == "High Risk" for d in fig.data]},
                           {"title": "Geographic Risk Scoring - High Risk"}]),
                dict(label="Medium Risk",
                     method="update",
                     args=[{"visible": [d.name == "Medium Risk" for d in fig.data]},
                           {"title": "Geographic Risk Scoring - Medium Risk"}]),
                dict(label="Low Risk",
                     method="update",
                     args=[{"visible": [d.name == "Low Risk" for d in fig.data]},
                           {"title": "Geographic Risk Scoring - Low Risk"}]),
            ]),
        )
    ])

fig.show()

Real time Response


In [56]:
import time

def real_time_response(anomalies, data):
    """
    Simulates real-time responses based on anomaly detection.

    Args:
        anomalies: A boolean array indicating anomalous data points.
        data: The original dataframe containing login information.
    """
    for i in range(len(anomalies)):
        if anomalies[i]:
            ip_address = map_data['IP Address'].iloc[i]
            timestamp = map_data['Login DateTime'].iloc[i]
            print(f"Anomaly detected at {timestamp} from IP: {ip_address}")

            # Block the IP address
            print(f"Blocking IP address: {ip_address}") # Integrate with a firewall or security system to block the IP.

            # Notify administrators
            print(f"Notifying administrators about suspicious activity from IP: {ip_address}") # Implement email/SMS/other notification mechanism.

            # Isolate affected resources
            print(f"Isolating resources associated with IP: {ip_address}")      # Session termination

            time.sleep(1)


real_time_response(anomalies[:100], map_data)

Anomaly detected at 2020-02-03 12:43:55.873000 from IP: 1369935930
Blocking IP address: 1369935930
Notifying administrators about suspicious activity from IP: 1369935930
Isolating resources associated with IP: 1369935930
Anomaly detected at 2020-02-03 12:43:59.396000 from IP: 167772207
Blocking IP address: 167772207
Notifying administrators about suspicious activity from IP: 167772207
Isolating resources associated with IP: 167772207
Anomaly detected at 2020-02-03 12:44:19.071000 from IP: 1558015394
Blocking IP address: 1558015394
Notifying administrators about suspicious activity from IP: 1558015394
Isolating resources associated with IP: 1558015394
Anomaly detected at 2020-02-03 12:44:24.849000 from IP: 2620665939
Blocking IP address: 2620665939
Notifying administrators about suspicious activity from IP: 2620665939
Isolating resources associated with IP: 2620665939
Anomaly detected at 2020-02-03 12:44:26.828000 from IP: 785658794
Blocking IP address: 785658794
Notifying administrator